# Last worked on 3/1/2018
# Goal reload cali_vphph_FILE with new numbers see end
# 2/28/2018 Save the DVRPC traffic count data as a PDF. Remove the bars with select
# object. Save as a .txt file. Do some regrex magic to change variable names and make them on their own line. Then just run read_n_record_cali_vph_file(cali_raw_TXTPATH,recNUM_LIST):
# read_n_save_Cali_runFILE(): can extract xml files and bring into a Python3 Pandas Dataframe object

# Next thing is to replace the XML values with new values
## re.sub(pattern, repl, string, max=0)
# https://www.tutorialspoint.com/python3/python_reg_expressions.htm

In [30]:
import numpy as np
import traci
import traci.constants as tc
import re
import time
import sumolib
# import cProfile, pstats , io
import pandas as pd
# import sumoPython_git_A as SP
import openpyxl as OPENxlsx
pd.set_option('display.max_columns', 99,'display.max_rows', 99)

In [93]:
cali_TEMP = pd.read_excel("C:\Dropbox\Phd_R_Ms\PhD_Modeling_DB_GIT\BMAOI_Dbox-DataFrames\Calibrator_Master_TEMPLATE_312018.xlsx")
PennDOT_AADT_day_month_TG3_DF = pd.read_excel("C:\Dropbox\Phd_R_Ms\Asset_Use_N_Management_Complete_Model\DropBox_ToolBox__MASTER__refresh_with_GIT\PennDOT AADT factors by day of week and month.xlsx")
PennDOT_Daily_Variance_TG3_DF = pd.read_excel("C:\Dropbox\Phd_R_Ms\Asset_Use_N_Management_Complete_Model\DropBox_ToolBox__MASTER__refresh_with_GIT\PennDOT vphph factors_TG3.xlsx")
# recNUM_LIST_OLD = ['33083', '33084', '33087', '37100', '37101', '37671', '37672', '47340', '47341', '47475', '53494', '53497', '53496', '53723', '53723', '67303', '67304', '78089', '87722', '87723', '110747', '110748', '110749', '110750', '118680', '127934', '127937', '135472', '135471', '135477', '135473', '135473', '118680']
recNUM_LIST = ['33083', '33084', '33087', '37100', '37101', '37671', '37672', '47340', '47341', '47475', '53494', '53497', '53496', '53723a', '53723', '67303', '67304', '78089', '87722', '87723', '110747', '110748', '110749', '110750', '118680a', '127934', '127937', '135472', '135471', '135477', '33087a','33087b','135473a', '135473b', '118680']
cali_raw_TXTPATH = '/Dropbox/Phd_R_Ms/SUMO_DB_root/Belmont_Ave-Run/DVRPC-Traffic_Count_Data-Belmount-AOI/Selected_Small_Belmount_AOI_ALL-class-turn-volume_MOD.txt'

In [86]:
cali_TEMP_DICT = dict()
for columns in cali_TEMP:
    if cali_TEMP.loc['RECORDNUM',columns] in recNUM_LIST:
        cali_TEMP_DICT.update({str(cali_TEMP.loc['RECORDNUM',columns]):str(columns)})
#         print(columns, cali_TEMP.loc['RECORDNUM',columns])
print(cali_TEMP_DICT)
print(cali_TEMP.columns)
recNum = '33083'
print(cali_TEMP_DICT[recNum])
# cali_TEMP.loc['RECORDNUM',30]# in recNUM_LIST
# cali_TEMP.iloc[cali_TEMP.index.get_loc('RECORDNUM'),int(cali_TEMP_DICT[recNum])] = 'pee'
# cali_TEMP
# cali_TEMP_DICT = {'37101': '4', '37672': '6', '53723': '14', '110748': '21', '47341': '8', '87723': '19', '33083': '0', '135471': '28', '110747': '20', '33087': '2', '53494': '10', '110749': '22', '53497': '11', '37100': '3', '33084': '1', '53496': '12', '78089': '17', '127934': '25', '110750': '23', '37671': '5', '47475': '9', '135472': '27', '127937': '26', '87722': '18', '47340': '7', '67304': '16', '118680': '34', '135473': '33', '67303': '15'}

{'37672': '6', '37101': '4', '110750': '23', '110748': '21', '37100': '3', '33087a': '30', '53723': '14', '53496': '12', '47341': '8', '33084': '1', '53494': '10', '127934': '25', '33087b': '31', '33083': '0', '47340': '7', '135477': '29', '135473a': '32', '37671': '5', '53497': '11', '135472': '27', '118680': '34', '87722': '18', '53723a': '13', '67304': '16', '118680a': '24', '47475': '9', '135473b': '33', '110747': '20', '135471': '28', '33087': '2', '67303': '15', '110749': '22', '87723': '19', '78089': '17', '127937': '26'}
Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34],
           dtype='int64')
0


In [87]:
def read_n_record_cali_vph_file(cali_raw_TXTPATH,recNUM_LIST):
    cali_TEMP_DICT = dict()
    for columns in cali_TEMP:
        if cali_TEMP.loc['RECORDNUM',columns] in recNUM_LIST:
            cali_TEMP_DICT.update({str(cali_TEMP.loc['RECORDNUM',columns]):str(columns)})
    #         print(columns, cali_TEMP.loc['RECORDNUM',columns])
    for recNum in recNUM_LIST:
#         counter +=1
        last_char = recNUM_LIST[int(cali_TEMP_DICT[recNum])][len(recNUM_LIST[int(cali_TEMP_DICT[recNum])])-1:len(recNUM_LIST[int(cali_TEMP_DICT[recNum])])]
        if re.search('[a-z]',last_char):
            recNuma = recNUM_LIST[int(cali_TEMP_DICT[recNum])][1:len(recNUM_LIST[int(cali_TEMP_DICT[recNum])])-1]
            print("Rerouting ",recNum," is now ", recNuma)
#         if recNum == '135473a': #'135473a', '135473b',
#             recNuma = '135473' #recNUM_LIST[recNum][1:len(recNUM_LIST[recNum])-1]
#         elif recNum == '135473b':
#             recNuma = '135473'
        else:
            recNuma = recNum
        with open(cali_raw_TXTPATH) as f:
            print("Searching for ",recNum)
            capture = 0
            for line in f:
                if recNuma in line:
#                     print(line)
                    capture = 1
                    AM = 1
                    PM = 0
                    day = 0
#                     counter += 1
                    print("\n<><> CAPTURING ", recNum, " and storing in column ", cali_TEMP_DICT[recNum])
                    RECORDNUM_LINE = line
                    RECORDNUM_ID = re.search("DVRPC_FILE_NUM: (.*)", RECORDNUM_LINE, re.IGNORECASE).group(1)
                    print("RECORDNUM_ID = ",RECORDNUM_ID, end="; ")
                    cali_TEMP.iloc[cali_TEMP.index.get_loc('rec_Check'),int(cali_TEMP_DICT[recNum])] = RECORDNUM_ID
#                     cali_TEMP.loc['rec_Check',cali_TEMP_DICT[recNum]] = RECORDNUM_ID
#                     cali_TEMP.loc['rec_Check',cali_TEMP['RECORDNUM'].str.contains(recNum)] = RECORDNUM_ID
#                     print("rec_Check = be true.. ", str(RECORDNUM_ID) == str(recNum),RECORDNUM_ID,"=?=",recNum)
#                 elif 'TO:' in line:
#                     TO_LINE = line
#                     TO_rd = re.search("TO: (.*)", TO_LINE, re.IGNORECASE).group(1)
#                     cali_TEMP.loc['TO',cali_TEMP_DICT[recNum]] = TO_rd
#                     print(TO_rd)
#                 elif 'FROM:' in line:
#                     FROM_LINE = line
#                     FROM_rd = re.search("FROM: (.*)", FROM_LINE, re.IGNORECASE).group(1)
#                     cali_TEMP.loc['FROM',cali_TEMP_DICT[recNum]] = FROM_rd   
#                     print(FROM_rd)
                if capture == 1:
                    if 'AXLE_CORR_FACTOR:' in line:
                        AXLE_CORR_FACTOR_LINE = line
                        AXLE_CORR_FACTOR_numb = re.search("AXLE_CORR_FACTOR: (.*)", AXLE_CORR_FACTOR_LINE, re.IGNORECASE).group(1)
#                         cali_TEMP.loc['AXLE_CORR_FACTOR',cali_TEMP_DICT[recNum]] = AXLE_CORR_FACTOR_numb
                        cali_TEMP.iloc[cali_TEMP.index.get_loc('AXLE_CORR_FACTOR'),int(cali_TEMP_DICT[recNum])] = AXLE_CORR_FACTOR_numb
                        print("AXLE_CORR_FACTOR_numb = ", AXLE_CORR_FACTOR_numb, end="; ")
                    elif 'Collected_on:' in line:
                        day += 1
                        Collected_on_LINE = line
                        Collected_on_DAY = re.search("Collected_on: (.*); ", Collected_on_LINE, re.IGNORECASE).group(1)
                        collected_on_t = "Collected_on_"+str(day) 
                        cali_TEMP.iloc[cali_TEMP.index.get_loc(collected_on_t),int(cali_TEMP_DICT[recNum])] = Collected_on_DAY
                        if day == 1:
                            Collected_on_DATE = re.search("Collected_on: (.*)\t(.*) ;", Collected_on_LINE, re.IGNORECASE).group(2)
                            cali_TEMP.iloc[cali_TEMP.index.get_loc('Collected_YR'),int(cali_TEMP_DICT[recNum])] = Collected_on_DATE
                            print("Collected Date = ",Collected_on_DATE,end="; ")
                    elif 'AADT:' in line:
                        AADT_LINE = line
                        AADT_numb = re.search("AADT: (.*)", AADT_LINE, re.IGNORECASE).group(1)
#                         cali_TEMP.loc['AADT',cali_TEMP_DICT[recNum]] = AADT_numb
                        cali_TEMP.iloc[cali_TEMP.index.get_loc('AADT'),int(cali_TEMP_DICT[recNum])] = AADT_numb
                        print("AADT = ", AADT_numb, end="; ")
                    elif 'AM_Peak_prct:' in line:
                        AM_Peak_prct_LINE = line
                        AM_Peak_prct_numb = re.search("AM_Peak_prct: (.*)", AM_Peak_prct_LINE, re.IGNORECASE).group(1)
#                         cali_TEMP.loc['AM_Peak_prct',cali_TEMP_DICT[recNum]] = AM_Peak_prct_numb
                        cali_TEMP.iloc[cali_TEMP.index.get_loc('AM_Peak_prct'),int(cali_TEMP_DICT[recNum])] = AM_Peak_prct_numb
                        print("AM_Peak_prct_numb = ",AM_Peak_prct_numb, end="; ")
                    elif 'Hour_Beginning:' in line:
                            Hour_Beginning_LINE = line
                            Hour_Beginning_numb = re.search("Hour_Beginning: (.*)", Hour_Beginning_LINE, re.IGNORECASE).group(1)
                            if Hour_Beginning_numb[len(Hour_Beginning_numb)-2:len(Hour_Beginning_numb)] == 'AM':
                                cali_TEMP.iloc[cali_TEMP.index.get_loc('Hour_Beginning_AM'),int(cali_TEMP_DICT[recNum])] = Hour_Beginning_numb
                                print("Hour_Beginning...AM = ",Hour_Beginning_numb, end="; ")
                            else:
                                cali_TEMP.iloc[cali_TEMP.index.get_loc('Hour_Beginning_PM'),int(cali_TEMP_DICT[recNum])] = Hour_Beginning_numb
                                print("Hour_Beginning...PM = ",Hour_Beginning_numb, end="\n")
                    elif 'SEASONAL_FACTOR:' in line:
                        SEASONAL_FACTOR_LINE = line
                        SEASONAL_FACTOR_numb = re.search("SEASONAL_FACTOR: (.*)", SEASONAL_FACTOR_LINE, re.IGNORECASE).group(1)
#                         cali_TEMP.loc['SEASONAL_FACTOR',cali_TEMP_DICT[recNum]] = SEASONAL_FACTOR_numb
                        cali_TEMP.iloc[cali_TEMP.index.get_loc('SEASONAL_FACTOR'),int(cali_TEMP_DICT[recNum])] = SEASONAL_FACTOR_numb
                        print("SEASONAL_FACTOR_numb = ",SEASONAL_FACTOR_numb, end="; ")
                    elif 'PM_Peak_prct:' in line:
                        PM_Peak_prct_LINE = line
                        PM_Peak_prct_numb = re.search("PM_Peak_prct: (.*)", PM_Peak_prct_LINE, re.IGNORECASE).group(1)
#                         cali_TEMP.loc['PM_Peak_prct',cali_TEMP_DICT[recNum]] = PM_Peak_prct_numb
                        cali_TEMP.iloc[cali_TEMP.index.get_loc('PM_Peak_prct'),int(cali_TEMP_DICT[recNum])] = PM_Peak_prct_numb
                        print("PM_Peak_prct_numb = ",PM_Peak_prct_numb, end="; ")
#                     elif 'Hour_Beginning:' in line:
#                         if PM == 1:
#                             Hour_Beginning_LINE =line
#                             Hour_Beginning_numb = re.search("Hour_Beginning: (.*)", Hour_Beginning_LINE, re.IGNORECASE).group(1)
#     #                         cali_TEMP.loc['Hour_Beginning',cali_TEMP_DICT[recNum]] = Hour_Beginning_numb
#                             cali_TEMP.iloc[cali_TEMP.index.get_loc('Hour_Beginning'),int(cali_TEMP_DICT[recNum])] = Hour_Beginning_numb
#                             print("Hour_Beginning...PM = ",Hour_Beginning_numb, end="\n")
# 
                    elif 'TAKEN BY' in line:
                        capture = 0
#                     counter += 1
                        print("\n\t\t\tabove was\n\t\t\t<><>",recNum,"<><>\n")# counter = ",counter,"\n")
    return cali_TEMP

In [88]:
# bigstring = open(cali_raw_TXTPATH)
# # bigstring.readlines()

In [89]:
cal_DF = read_n_record_cali_vph_file(cali_raw_TXTPATH,recNUM_LIST)

Searching for  33083

<><> CAPTURING  33083  and storing in column  0
RECORDNUM_ID =  33083 ; Collected Date =  12/2/2002; AXLE_CORR_FACTOR_numb =  0.987; AADT =  1,024; AM_Peak_prct_numb =  20.3; Hour_Beginning...AM =  8:00 AM; SEASONAL_FACTOR_numb =  0.994 ; PM_Peak_prct_numb =  6.0 ; Hour_Beginning...PM =  4:00 PM

			above was
			<><> 33083 <><>

Searching for  33084

<><> CAPTURING  33084  and storing in column  1
RECORDNUM_ID =  33084 ; Collected Date =  12/2/2002; AXLE_CORR_FACTOR_numb =  0.987; AADT =  1,039; AM_Peak_prct_numb =  7.3; Hour_Beginning...AM =  7:00 AM; SEASONAL_FACTOR_numb =  0.994 ; PM_Peak_prct_numb =  10.0 ; Hour_Beginning...PM =  5:00 PM

			above was
			<><> 33084 <><>

Searching for  33087

<><> CAPTURING  33087  and storing in column  2
RECORDNUM_ID =  33087 ; Collected Date =  12/2/2002; AXLE_CORR_FACTOR_numb =  0.987; AADT =  1,993; AM_Peak_prct_numb =  10.5; Hour_Beginning...AM =  7:00 AM; SEASONAL_FACTOR_numb =  0.994 ; PM_Peak_prct_numb =  8.9 ; Hour_B

In [90]:
pd.DataFrame.to_excel(cali_TEMP,'C:\Dropbox\Phd_R_Ms\PhD_Modeling_DB_GIT\BMAOI_Dbox-DataFrames\Calibrator_Master_V3.xlsx')
# 

In [96]:
cali_M_DF

NameError: name 'cali_M_DF' is not defined

In [95]:
PennDOT_AADT_day_month_TG3_DF
# PennDOT_Daily_Variance_TG3_DF

,January,February,March,April,May,June,July,August,September,October,November,December
Monday,1.030,1.070,0.986,0.926,1.037,0.906,0.930,0.917,1.016,0.940,0.940,0.941
Tuesday,1.072,1.003,0.932,0.957,0.874,0.875,0.889,0.890,0.889,0.893,0.897,0.926
Wednesday,1.129,1.084,0.922,0.903,0.863,0.860,0.871,0.869,0.869,0.887,0.914,0.939
Thursday,0.961,1.039,0.903,0.869,0.858,0.857,0.863,0.961,0.862,0.869,0.952,1.087
Friday,0.953,0.963,0.867,0.881,0.838,0.840,0.987,0.931,0.838,0.851,0.939,0.920
Saturday,1.285,1.218,1.107,1.075,1.079,1.075,1.155,1.113,1.079,1.109,1.143,1.124
Sunday,1.546,1.509,1.318,1.263,1.270,1.234,1.310,1.325,1.264,1.264,1.366,1.379
DAY_of_Month,1.139,1.127,1.005,0.982,0.974,0.950,1.001,1.001,0.974,0.973,1.022,1.045


In [ ]:
    def read_n_save_Cali_runFILE():
        SUMO_outPUT_PREFIX = "TESTii"
        PATH_cali_MASTER_TEMPLATExlsx = '/Dropbox/Phd_R_Ms/Asset_Use_N_Management_Complete_Model/DropBox_ToolBox__MASTER__refresh_with_GIT/Calibrator_Master_TEMPLATE.xlsx'
        PATH_to_Save_to = "/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/BMAOI_Dbox-DataFrames/Network_" + SUMO_outPUT_PREFIX + "_Calibrator_XML_extract.xlsx"
        logger_TEMPDF =pd.read_excel(PATH_cali_MASTER_TEMPLATExlsx)
#         global SUMO_outPUT_PREFIX
#         global SUMO_Traci_PORT
        counter = 0
        readfromPATH = '/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/Belmont_AOI_git/Belmont_AOI_toolBOX/Belmount_AOI-caliborators-DB-V6-SUMO31-15min_RAW.xml'
        with open(readfromPATH) as f:
        #open('/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/Belmont_AOI_git/Belmont_AOI-runFILES/BMAOI_TRACI_DB.sumocfg') as f:
            termLIST = list()
            termcounter = 0
            for line in f:
                # print("SUMO_outPUT_PREFIX = ",SUMO_outPUT_PREFIX)
                # if SUMO_outPUT_PREFIX =="":
#                 routeProbe id="ConcrseDR-Belmont-MemHll-EB-02-RouteProbe" edge="-106455704#9"
                if '<routeProbe id="' in line:
                    counter += 1
#                     print(line)
                    routeProbe_ID_LINE = line
                    routeProbe_ID = re.search('<routeProbe id="(.*)" e',routeProbe_ID_LINE, re.IGNORECASE).group(1)
                    termLIST.append(re.search('<routeProbe id="(.*)" e',routeProbe_ID_LINE, re.IGNORECASE).group(1))
                    logger_TEMPDF.loc['routeProbe_ID',counter-1] = routeProbe_ID
#                     routeProbe_ID = termLIST[0]
                    # print("\nSUMO_outPUT_PREFIX = ",SUMO_outPUT_PREFIX,"\n")
                    termcounter +=1
                elif '<calibrator id=' in line:
                    calibrator_ID_LINE = line
                    calibrator_ID = re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(1)
                    termLIST.append(re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(1))
                    calibrator_ID_edge = re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(2)
                    termLIST.append(re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(2))
                    logger_TEMPDF.loc['calibrator_ID',counter-1] = calibrator_ID
                    logger_TEMPDF.loc['SUMO_edge_ID',counter-1] = calibrator_ID_edge
#                 elif '<flow begin="1"' in line:
# #                     print(line)
# #                     print(re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', line, re.IGNORECASE).group(3))
#                     flow_vph_1_ID_LINE = line
#                     flow_vph_1_ID = re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_1_ID_LINE, re.IGNORECASE).group(3)
#                     termLIST.append(re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_1_ID_LINE, re.IGNORECASE).group(3))
#                     logger_TEMPDF.loc['vph_start_1',counter-1] = flow_vph_1_ID
                elif '<flow begin="1"' in line:
                    flow_vph_1_ID_LINE = line
                    flow_vph_1_ID = re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_1_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_1_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_1', counter-1] = flow_vph_1_ID
                elif '<flow begin="901"' in line:
                    flow_vph_901_ID_LINE = line
                    flow_vph_901_ID = re.search('<flow begin="901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_901', counter-1] = flow_vph_901_ID
                elif '<flow begin="1801"' in line:
                    flow_vph_1801_ID_LINE = line
                    flow_vph_1801_ID = re.search('<flow begin="1801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_1801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="1801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_1801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_1801', counter-1] = flow_vph_1801_ID
                elif '<flow begin="2701"' in line:
                    flow_vph_2701_ID_LINE = line
                    flow_vph_2701_ID = re.search('<flow begin="2701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_2701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="2701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_2701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_2701', counter-1] = flow_vph_2701_ID
                elif '<flow begin="3601"' in line:
                    flow_vph_3601_ID_LINE = line
                    flow_vph_3601_ID = re.search('<flow begin="3601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_3601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="3601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_3601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_3601', counter-1] = flow_vph_3601_ID
                elif '<flow begin="4501"' in line:
                    flow_vph_4501_ID_LINE = line
                    flow_vph_4501_ID = re.search('<flow begin="4501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_4501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="4501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_4501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_4501', counter-1] = flow_vph_4501_ID
                elif '<flow begin="5401"' in line:
                    flow_vph_5401_ID_LINE = line
                    flow_vph_5401_ID = re.search('<flow begin="5401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_5401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="5401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_5401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_5401', counter-1] = flow_vph_5401_ID
                elif '<flow begin="6301"' in line:
                    flow_vph_6301_ID_LINE = line
                    flow_vph_6301_ID = re.search('<flow begin="6301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_6301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="6301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_6301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_6301', counter-1] = flow_vph_6301_ID
                elif '<flow begin="7201"' in line:
                    flow_vph_7201_ID_LINE = line
                    flow_vph_7201_ID = re.search('<flow begin="7201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_7201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="7201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_7201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_7201', counter-1] = flow_vph_7201_ID
                elif '<flow begin="8101"' in line:
                    flow_vph_8101_ID_LINE = line
                    flow_vph_8101_ID = re.search('<flow begin="8101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_8101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="8101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_8101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_8101', counter-1] = flow_vph_8101_ID
                elif '<flow begin="9001"' in line:
                    flow_vph_9001_ID_LINE = line
                    flow_vph_9001_ID = re.search('<flow begin="9001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_9001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="9001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_9001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_9001', counter-1] = flow_vph_9001_ID
                elif '<flow begin="9901"' in line:
                    flow_vph_9901_ID_LINE = line
                    flow_vph_9901_ID = re.search('<flow begin="9901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_9901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="9901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_9901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_9901', counter-1] = flow_vph_9901_ID
                elif '<flow begin="10801"' in line:
                    flow_vph_10801_ID_LINE = line
                    flow_vph_10801_ID = re.search('<flow begin="10801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_10801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="10801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_10801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_10801', counter-1] = flow_vph_10801_ID
                elif '<flow begin="11701"' in line:
                    flow_vph_11701_ID_LINE = line
                    flow_vph_11701_ID = re.search('<flow begin="11701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_11701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="11701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_11701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_11701', counter-1] = flow_vph_11701_ID
                elif '<flow begin="12601"' in line:
                    flow_vph_12601_ID_LINE = line
                    flow_vph_12601_ID = re.search('<flow begin="12601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_12601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="12601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_12601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_12601', counter-1] = flow_vph_12601_ID
                elif '<flow begin="13501"' in line:
                    flow_vph_13501_ID_LINE = line
                    flow_vph_13501_ID = re.search('<flow begin="13501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_13501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="13501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_13501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_13501', counter-1] = flow_vph_13501_ID
                elif '<flow begin="14401"' in line:
                    flow_vph_14401_ID_LINE = line
                    flow_vph_14401_ID = re.search('<flow begin="14401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_14401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="14401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_14401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_14401', counter-1] = flow_vph_14401_ID
                elif '<flow begin="15301"' in line:
                    flow_vph_15301_ID_LINE = line
                    flow_vph_15301_ID = re.search('<flow begin="15301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_15301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="15301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_15301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_15301', counter-1] = flow_vph_15301_ID
                elif '<flow begin="16201"' in line:
                    flow_vph_16201_ID_LINE = line
                    flow_vph_16201_ID = re.search('<flow begin="16201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_16201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="16201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_16201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_16201', counter-1] = flow_vph_16201_ID
                elif '<flow begin="17101"' in line:
                    flow_vph_17101_ID_LINE = line
                    flow_vph_17101_ID = re.search('<flow begin="17101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_17101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="17101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_17101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_17101', counter-1] = flow_vph_17101_ID
                elif '<flow begin="18001"' in line:
                    flow_vph_18001_ID_LINE = line
                    flow_vph_18001_ID = re.search('<flow begin="18001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_18001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="18001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_18001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_18001', counter-1] = flow_vph_18001_ID
                elif '<flow begin="18901"' in line:
                    flow_vph_18901_ID_LINE = line
                    flow_vph_18901_ID = re.search('<flow begin="18901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_18901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="18901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_18901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_18901', counter-1] = flow_vph_18901_ID
                elif '<flow begin="19801"' in line:
                    flow_vph_19801_ID_LINE = line
                    flow_vph_19801_ID = re.search('<flow begin="19801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_19801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="19801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_19801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_19801', counter-1] = flow_vph_19801_ID
                elif '<flow begin="20701"' in line:
                    flow_vph_20701_ID_LINE = line
                    flow_vph_20701_ID = re.search('<flow begin="20701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_20701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="20701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_20701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_20701', counter-1] = flow_vph_20701_ID
                elif '<flow begin="21601"' in line:
                    flow_vph_21601_ID_LINE = line
                    flow_vph_21601_ID = re.search('<flow begin="21601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_21601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="21601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_21601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_21601', counter-1] = flow_vph_21601_ID
                elif '<flow begin="22501"' in line:
                    flow_vph_22501_ID_LINE = line
                    flow_vph_22501_ID = re.search('<flow begin="22501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_22501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="22501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_22501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_22501', counter-1] = flow_vph_22501_ID
                elif '<flow begin="23401"' in line:
                    flow_vph_23401_ID_LINE = line
                    flow_vph_23401_ID = re.search('<flow begin="23401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_23401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="23401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_23401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_23401', counter-1] = flow_vph_23401_ID
                elif '<flow begin="24301"' in line:
                    flow_vph_24301_ID_LINE = line
                    flow_vph_24301_ID = re.search('<flow begin="24301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_24301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="24301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_24301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_24301', counter-1] = flow_vph_24301_ID
                elif '<flow begin="25201"' in line:
                    flow_vph_25201_ID_LINE = line
                    flow_vph_25201_ID = re.search('<flow begin="25201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_25201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="25201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_25201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_25201', counter-1] = flow_vph_25201_ID
                elif '<flow begin="26101"' in line:
                    flow_vph_26101_ID_LINE = line
                    flow_vph_26101_ID = re.search('<flow begin="26101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_26101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="26101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_26101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_26101', counter-1] = flow_vph_26101_ID
                elif '<flow begin="27001"' in line:
                    flow_vph_27001_ID_LINE = line
                    flow_vph_27001_ID = re.search('<flow begin="27001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_27001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="27001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_27001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_27001', counter-1] = flow_vph_27001_ID
                elif '<flow begin="27901"' in line:
                    flow_vph_27901_ID_LINE = line
                    flow_vph_27901_ID = re.search('<flow begin="27901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_27901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="27901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_27901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_27901', counter-1] = flow_vph_27901_ID
                elif '<flow begin="28801"' in line:
                    flow_vph_28801_ID_LINE = line
                    flow_vph_28801_ID = re.search('<flow begin="28801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_28801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="28801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_28801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_28801', counter-1] = flow_vph_28801_ID
                elif '<flow begin="29701"' in line:
                    flow_vph_29701_ID_LINE = line
                    flow_vph_29701_ID = re.search('<flow begin="29701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_29701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="29701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_29701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_29701', counter-1] = flow_vph_29701_ID
                elif '<flow begin="30601"' in line:
                    flow_vph_30601_ID_LINE = line
                    flow_vph_30601_ID = re.search('<flow begin="30601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_30601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="30601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_30601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_30601', counter-1] = flow_vph_30601_ID
                elif '<flow begin="31501"' in line:
                    flow_vph_31501_ID_LINE = line
                    flow_vph_31501_ID = re.search('<flow begin="31501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_31501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="31501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_31501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_31501', counter-1] = flow_vph_31501_ID
                elif '<flow begin="32401"' in line:
                    flow_vph_32401_ID_LINE = line
                    flow_vph_32401_ID = re.search('<flow begin="32401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_32401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="32401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_32401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_32401', counter-1] = flow_vph_32401_ID
                elif '<flow begin="33301"' in line:
                    flow_vph_33301_ID_LINE = line
                    flow_vph_33301_ID = re.search('<flow begin="33301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_33301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="33301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_33301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_33301', counter-1] = flow_vph_33301_ID
                elif '<flow begin="34201"' in line:
                    flow_vph_34201_ID_LINE = line
                    flow_vph_34201_ID = re.search('<flow begin="34201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_34201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="34201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_34201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_34201', counter-1] = flow_vph_34201_ID
                elif '<flow begin="35101"' in line:
                    flow_vph_35101_ID_LINE = line
                    flow_vph_35101_ID = re.search('<flow begin="35101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_35101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="35101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_35101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_35101', counter-1] = flow_vph_35101_ID
                elif '<flow begin="36001"' in line:
                    flow_vph_36001_ID_LINE = line
                    flow_vph_36001_ID = re.search('<flow begin="36001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_36001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="36001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_36001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_36001', counter-1] = flow_vph_36001_ID
                elif '<flow begin="36901"' in line:
                    flow_vph_36901_ID_LINE = line
                    flow_vph_36901_ID = re.search('<flow begin="36901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_36901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="36901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_36901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_36901', counter-1] = flow_vph_36901_ID
                elif '<flow begin="37801"' in line:
                    flow_vph_37801_ID_LINE = line
                    flow_vph_37801_ID = re.search('<flow begin="37801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_37801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="37801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_37801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_37801', counter-1] = flow_vph_37801_ID
                elif '<flow begin="38701"' in line:
                    flow_vph_38701_ID_LINE = line
                    flow_vph_38701_ID = re.search('<flow begin="38701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_38701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="38701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_38701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_38701', counter-1] = flow_vph_38701_ID
                elif '<flow begin="39601"' in line:
                    flow_vph_39601_ID_LINE = line
                    flow_vph_39601_ID = re.search('<flow begin="39601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_39601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="39601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_39601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_39601', counter-1] = flow_vph_39601_ID
                elif '<flow begin="40501"' in line:
                    flow_vph_40501_ID_LINE = line
                    flow_vph_40501_ID = re.search('<flow begin="40501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_40501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="40501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_40501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_40501', counter-1] = flow_vph_40501_ID
                elif '<flow begin="41401"' in line:
                    flow_vph_41401_ID_LINE = line
                    flow_vph_41401_ID = re.search('<flow begin="41401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_41401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="41401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_41401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_41401', counter-1] = flow_vph_41401_ID
                elif '<flow begin="42301"' in line:
                    flow_vph_42301_ID_LINE = line
                    flow_vph_42301_ID = re.search('<flow begin="42301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_42301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="42301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_42301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_42301', counter-1] = flow_vph_42301_ID
                elif '<flow begin="43201"' in line:
                    flow_vph_43201_ID_LINE = line
                    flow_vph_43201_ID = re.search('<flow begin="43201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_43201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="43201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_43201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_43201', counter-1] = flow_vph_43201_ID
                elif '<flow begin="44101"' in line:
                    flow_vph_44101_ID_LINE = line
                    flow_vph_44101_ID = re.search('<flow begin="44101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_44101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="44101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_44101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_44101', counter-1] = flow_vph_44101_ID
                elif '<flow begin="45001"' in line:
                    flow_vph_45001_ID_LINE = line
                    flow_vph_45001_ID = re.search('<flow begin="45001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_45001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="45001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_45001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_45001', counter-1] = flow_vph_45001_ID
                elif '<flow begin="45901"' in line:
                    flow_vph_45901_ID_LINE = line
                    flow_vph_45901_ID = re.search('<flow begin="45901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_45901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="45901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_45901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_45901', counter-1] = flow_vph_45901_ID
                elif '<flow begin="46801"' in line:
                    flow_vph_46801_ID_LINE = line
                    flow_vph_46801_ID = re.search('<flow begin="46801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_46801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="46801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_46801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_46801', counter-1] = flow_vph_46801_ID
                elif '<flow begin="47701"' in line:
                    flow_vph_47701_ID_LINE = line
                    flow_vph_47701_ID = re.search('<flow begin="47701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_47701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="47701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_47701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_47701', counter-1] = flow_vph_47701_ID
                elif '<flow begin="48601"' in line:
                    flow_vph_48601_ID_LINE = line
                    flow_vph_48601_ID = re.search('<flow begin="48601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_48601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="48601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_48601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_48601', counter-1] = flow_vph_48601_ID
                elif '<flow begin="49501"' in line:
                    flow_vph_49501_ID_LINE = line
                    flow_vph_49501_ID = re.search('<flow begin="49501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_49501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="49501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_49501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_49501', counter-1] = flow_vph_49501_ID
                elif '<flow begin="50401"' in line:
                    flow_vph_50401_ID_LINE = line
                    flow_vph_50401_ID = re.search('<flow begin="50401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_50401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="50401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_50401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_50401', counter-1] = flow_vph_50401_ID
                elif '<flow begin="51301"' in line:
                    flow_vph_51301_ID_LINE = line
                    flow_vph_51301_ID = re.search('<flow begin="51301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_51301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="51301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_51301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_51301', counter-1] = flow_vph_51301_ID
                elif '<flow begin="52201"' in line:
                    flow_vph_52201_ID_LINE = line
                    flow_vph_52201_ID = re.search('<flow begin="52201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_52201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="52201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_52201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_52201', counter-1] = flow_vph_52201_ID
                elif '<flow begin="53101"' in line:
                    flow_vph_53101_ID_LINE = line
                    flow_vph_53101_ID = re.search('<flow begin="53101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_53101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="53101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_53101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_53101', counter-1] = flow_vph_53101_ID
                elif '<flow begin="54001"' in line:
                    flow_vph_54001_ID_LINE = line
                    flow_vph_54001_ID = re.search('<flow begin="54001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_54001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="54001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_54001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_54001', counter-1] = flow_vph_54001_ID
                elif '<flow begin="54901"' in line:
                    flow_vph_54901_ID_LINE = line
                    flow_vph_54901_ID = re.search('<flow begin="54901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_54901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="54901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_54901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_54901', counter-1] = flow_vph_54901_ID
                elif '<flow begin="55801"' in line:
                    flow_vph_55801_ID_LINE = line
                    flow_vph_55801_ID = re.search('<flow begin="55801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_55801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="55801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_55801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_55801', counter-1] = flow_vph_55801_ID
                elif '<flow begin="56701"' in line:
                    flow_vph_56701_ID_LINE = line
                    flow_vph_56701_ID = re.search('<flow begin="56701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_56701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="56701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_56701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_56701', counter-1] = flow_vph_56701_ID
                elif '<flow begin="57601"' in line:
                    flow_vph_57601_ID_LINE = line
                    flow_vph_57601_ID = re.search('<flow begin="57601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_57601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="57601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_57601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_57601', counter-1] = flow_vph_57601_ID
                elif '<flow begin="58501"' in line:
                    flow_vph_58501_ID_LINE = line
                    flow_vph_58501_ID = re.search('<flow begin="58501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_58501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="58501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_58501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_58501', counter-1] = flow_vph_58501_ID
                elif '<flow begin="59401"' in line:
                    flow_vph_59401_ID_LINE = line
                    flow_vph_59401_ID = re.search('<flow begin="59401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_59401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="59401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_59401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_59401', counter-1] = flow_vph_59401_ID
                elif '<flow begin="60301"' in line:
                    flow_vph_60301_ID_LINE = line
                    flow_vph_60301_ID = re.search('<flow begin="60301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_60301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="60301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_60301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_60301', counter-1] = flow_vph_60301_ID
                elif '<flow begin="61201"' in line:
                    flow_vph_61201_ID_LINE = line
                    flow_vph_61201_ID = re.search('<flow begin="61201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_61201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="61201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_61201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_61201', counter-1] = flow_vph_61201_ID
                elif '<flow begin="62101"' in line:
                    flow_vph_62101_ID_LINE = line
                    flow_vph_62101_ID = re.search('<flow begin="62101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_62101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="62101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_62101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_62101', counter-1] = flow_vph_62101_ID
                elif '<flow begin="63001"' in line:
                    flow_vph_63001_ID_LINE = line
                    flow_vph_63001_ID = re.search('<flow begin="63001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_63001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="63001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_63001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_63001', counter-1] = flow_vph_63001_ID
                elif '<flow begin="63901"' in line:
                    flow_vph_63901_ID_LINE = line
                    flow_vph_63901_ID = re.search('<flow begin="63901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_63901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="63901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_63901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_63901', counter-1] = flow_vph_63901_ID
                elif '<flow begin="64801"' in line:
                    flow_vph_64801_ID_LINE = line
                    flow_vph_64801_ID = re.search('<flow begin="64801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_64801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="64801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_64801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_64801', counter-1] = flow_vph_64801_ID
                elif '<flow begin="65701"' in line:
                    flow_vph_65701_ID_LINE = line
                    flow_vph_65701_ID = re.search('<flow begin="65701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_65701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="65701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_65701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_65701', counter-1] = flow_vph_65701_ID
                elif '<flow begin="66601"' in line:
                    flow_vph_66601_ID_LINE = line
                    flow_vph_66601_ID = re.search('<flow begin="66601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_66601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="66601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_66601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_66601', counter-1] = flow_vph_66601_ID
                elif '<flow begin="67501"' in line:
                    flow_vph_67501_ID_LINE = line
                    flow_vph_67501_ID = re.search('<flow begin="67501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_67501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="67501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_67501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_67501', counter-1] = flow_vph_67501_ID
                elif '<flow begin="68401"' in line:
                    flow_vph_68401_ID_LINE = line
                    flow_vph_68401_ID = re.search('<flow begin="68401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_68401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="68401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_68401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_68401', counter-1] = flow_vph_68401_ID
                elif '<flow begin="69301"' in line:
                    flow_vph_69301_ID_LINE = line
                    flow_vph_69301_ID = re.search('<flow begin="69301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_69301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="69301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_69301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_69301', counter-1] = flow_vph_69301_ID
                elif '<flow begin="70201"' in line:
                    flow_vph_70201_ID_LINE = line
                    flow_vph_70201_ID = re.search('<flow begin="70201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_70201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="70201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_70201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_70201', counter-1] = flow_vph_70201_ID
                elif '<flow begin="71101"' in line:
                    flow_vph_71101_ID_LINE = line
                    flow_vph_71101_ID = re.search('<flow begin="71101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_71101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="71101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_71101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_71101', counter-1] = flow_vph_71101_ID
                elif '<flow begin="72001"' in line:
                    flow_vph_72001_ID_LINE = line
                    flow_vph_72001_ID = re.search('<flow begin="72001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_72001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="72001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_72001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_72001', counter-1] = flow_vph_72001_ID
                elif '<flow begin="72901"' in line:
                    flow_vph_72901_ID_LINE = line
                    flow_vph_72901_ID = re.search('<flow begin="72901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_72901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="72901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_72901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_72901', counter-1] = flow_vph_72901_ID
                elif '<flow begin="73801"' in line:
                    flow_vph_73801_ID_LINE = line
                    flow_vph_73801_ID = re.search('<flow begin="73801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_73801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="73801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_73801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_73801', counter-1] = flow_vph_73801_ID
                elif '<flow begin="74701"' in line:
                    flow_vph_74701_ID_LINE = line
                    flow_vph_74701_ID = re.search('<flow begin="74701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_74701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="74701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_74701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_74701', counter-1] = flow_vph_74701_ID
                elif '<flow begin="75601"' in line:
                    flow_vph_75601_ID_LINE = line
                    flow_vph_75601_ID = re.search('<flow begin="75601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_75601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="75601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_75601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_75601', counter-1] = flow_vph_75601_ID
                elif '<flow begin="76501"' in line:
                    flow_vph_76501_ID_LINE = line
                    flow_vph_76501_ID = re.search('<flow begin="76501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_76501_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="76501" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_76501_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_76501', counter-1] = flow_vph_76501_ID
                elif '<flow begin="77401"' in line:
                    flow_vph_77401_ID_LINE = line
                    flow_vph_77401_ID = re.search('<flow begin="77401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_77401_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="77401" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_77401_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_77401', counter-1] = flow_vph_77401_ID
                elif '<flow begin="78301"' in line:
                    flow_vph_78301_ID_LINE = line
                    flow_vph_78301_ID = re.search('<flow begin="78301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_78301_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="78301" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_78301_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_78301', counter-1] = flow_vph_78301_ID
                elif '<flow begin="79201"' in line:
                    flow_vph_79201_ID_LINE = line
                    flow_vph_79201_ID = re.search('<flow begin="79201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_79201_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="79201" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_79201_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_79201', counter-1] = flow_vph_79201_ID
                elif '<flow begin="80101"' in line:
                    flow_vph_80101_ID_LINE = line
                    flow_vph_80101_ID = re.search('<flow begin="80101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_80101_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="80101" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_80101_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_80101', counter-1] = flow_vph_80101_ID
                elif '<flow begin="81001"' in line:
                    flow_vph_81001_ID_LINE = line
                    flow_vph_81001_ID = re.search('<flow begin="81001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_81001_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="81001" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_81001_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_81001', counter-1] = flow_vph_81001_ID
                elif '<flow begin="81901"' in line:
                    flow_vph_81901_ID_LINE = line
                    flow_vph_81901_ID = re.search('<flow begin="81901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_81901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="81901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_81901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_81901', counter-1] = flow_vph_81901_ID
                elif '<flow begin="82801"' in line:
                    flow_vph_82801_ID_LINE = line
                    flow_vph_82801_ID = re.search('<flow begin="82801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_82801_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="82801" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_82801_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_82801', counter-1] = flow_vph_82801_ID
                elif '<flow begin="83701"' in line:
                    flow_vph_83701_ID_LINE = line
                    flow_vph_83701_ID = re.search('<flow begin="83701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_83701_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="83701" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_83701_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_83701', counter-1] = flow_vph_83701_ID
                elif '<flow begin="84601"' in line:
                    flow_vph_84601_ID_LINE = line
                    flow_vph_84601_ID = re.search('<flow begin="84601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_84601_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="84601" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_84601_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_84601', counter-1] = flow_vph_84601_ID
        pd.DataFrame.to_excel(logger_TEMPDF,'/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/BMAOI_Dbox-DataFrames/Calibrator_files_v1.xlsx')
#                     print(routeProbe_ID, calibrator_ID, calibrator_ID_edge, "vph =",flow_vph_1_ID)

        return logger_TEMPDF #routeProbe_ID, calibrator_ID,calibrator_ID_edge#, print("\nSUMO_outPUT_PREFIX = ", SUMO_outPUT_PREFIX,"\nSUMO_Traci_PORT = ", SUMO_Traci_PORT)

In [ ]:
read_n_save_Cali_runFILE()


        PATH_cali_MASTER_TEMPLATExlsx = '/Dropbox/Phd_R_Ms/Asset_Use_N_Management_Complete_Model/DropBox_ToolBox__MASTER__refresh_with_GIT/Calibrator_Master_TEMPLATE.xlsx'
        PATH_to_Save_to = "/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/BMAOI_Dbox-DataFrames/Network_" + SUMO_outPUT_PREFIX + "_PeriodBook.xlsx"
        wb =  OPENxlsx.load_workbook(filename = PATH_to_Save_to)

In [32]:
cali_M_DF = pd.read_excel("C:\Dropbox\Phd_R_Ms\PhD_Modeling_DB_GIT\BMAOI_Dbox-DataFrames\Calibrator_Master_TEMPLATE_312018.xlsx")
readfromPATH = '/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/Belmont_AOI_git/Belmont_AOI_toolBOX_Not_Master/Belmount_AOI-caliborators-DB-V6-SUMO31-15min_RAW.xml'
caliXMLTESTPATH = '/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/Belmont_AOI_git/Belmont_AOI_toolBOX_Not_Master/cali_test.xml'
print(cali_M_DF.shape,cali_M_DF.columns, cali_M_DF.index)

(107, 35) Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34],
           dtype='int64') Index(['calibrator_ID', 'routeProbe_ID', 'SUMO_edge_ID', 'vph_start_1',
       'vph_start_901', 'vph_start_1801', 'vph_start_2701', 'vph_start_3601',
       'vph_start_4501', 'vph_start_5401',
       ...
       'vph_start_84601', 'RECORDNUM', 'AXLE_CORR_FACTOR', 'AADT',
       'AM_Peak_prct', 'Hour_Beginning_AM', 'SEASONAL_FACTOR', 'PM_Peak_prct',
       'Hour_Beginning_PM', 'rec_Check'],
      dtype='object', length=107)


In [ ]:
## I don't think I will be able to replace in kind, but I should be able to write each line

In [84]:
XML_cali = open(caliXMLTESTPATH,'r')
XMLdata = XML_cali.read()
XML_cali.close()
capture = 0
for line in XMLdata:
    if '<routeProbe id="' in line:
        routeProbe_ID_LINE = line
        routeProbe_ID = re.search('<routeProbe id="(.*)" e',routeProbe_ID_LINE, re.IGNORECASE).group(1)
        for cols in range(col_cali):
            if cali_M_DF.iloc[1,cols] == routeProbe_ID:
                col_DF = cols
                capture = 1
                break
    if capture == 1:
        if '<flow begin="1"' in line:
                print(line)
                flow_vph_1_row = 3
                flow_vph_1_ID_LINE = line
                flow_vph_1_ID = 10#cali_M_DF.iloc[flow_vph_1_row,col_DF]# * 1.33) # This needs tp be a function
                replace_Value = 'vehsPerHour="'+"Poo"+'" t'

                pp = (re.sub('vehsPerHour="(.*)" t',replace_Value, flow_vph_1_ID_LINE, re.IGNORECASE))
                XMLdata.line.replace(flow_vph_1_ID_LINE,pp)
                capture = 0
                print("flow_vph_1 being replaced ", replace_Value, "col_DF = ",col_DF,flow_vph_1_ID_LINE)
XML_cali = open(caliXMLTESTPATH,'w')
XML_cali.writelines(XMLdata)
XML_cali.close()

In [85]:
XML_cali = open(caliXMLTESTPATH,'r')
XMLdata = XML_cali.read()
for lines in XMLdata:
    if '<routeProbe id="' in lines:
        routeProbe_ID_LINE = lines
        routeProbe_ID = re.search('<routeProbe id="(.*)" e',routeProbe_ID_LINE, re.IGNORECASE).group(1)
        for cols in range(col_cali):
            if cali_M_DF.iloc[1,cols] == routeProbe_ID:
                col_DF = cols
                capture = 1
                break
    if capture == 1:
        if '<flow begin="1"' in line:
            flow_vph_1_ID_LINE = line
            flow_vph_1_ID = re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_1_ID_LINE, re.IGNORECASE).group(3)
            re.sub(flow_vph_1_ID,repl="poo",string=line)
XML_cali.close()

In [82]:
xmlrep,XMLdata

('\t<flow begin="1" end="900" route="ConcrseDR-Belmont-MemHll-WB-02-Distrib" vehsPerHour="poo" type="US1-ALL-NB_Dis" departPos="free" departSpeed="max"/>\n',
 '\t<flow begin="1" end="900" route="ConcrseDR-Belmont-MemHll-EB-02-Distrib" vehsPerHour="poo" type="US1-ALL-NB_Dis" departPos="free" departSpeed="max"/>\n')

In [33]:
row_cali = cali_M_DF.shape[0]
col_cali = cali_M_DF.shape[1]
with open(caliXMLTESTPATH,'r') as f:
    capture = 0
    for line in f:
        if '<routeProbe id="' in line:
            routeProbe_ID_LINE = line
            routeProbe_ID = re.search('<routeProbe id="(.*)" e',routeProbe_ID_LINE, re.IGNORECASE).group(1)
            for cols in range(col_cali):
                if cali_M_DF.iloc[1,cols] == routeProbe_ID:
                    col_DF = cols
                    capture = 1
                    break
        if capture == 1:
            if '<calibrator id=' in line:
                calibrator_ID_LINE = line
                calibrator_ID = re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(1)
                if calibrator_ID == cali_M_DF.iloc[0,col_DF]:
                    print("YAY!!!")
                else:
                    print("Shit...")
            elif '<flow begin="1"' in line:
                flow_vph_1_row = 3
                flow_vph_1_ID_LINE = line
                flow_vph_1_ID = "POO"#str(flow_vph_1_ID_LINE,cali_M_DF.iloc[flow_vph_1_row,col_DF])# * 1.33) # This needs tp be a function
                re.sub('vehsPerHour="(.*)" t','vehsPerHour=",flow_vph_1_ID," t', calibrator_ID_LINE, re.IGNORECASE)
                capture = 0
                print("flow_vph_1 being replaced ", line)
#         print(capture)



YAY!!!
flow_vph_1 being replaced  	<flow begin="1" end="900" route="ConcrseDR-Belmont-MemHll-EB-02-Distrib" vehsPerHour="5" type="US1-ALL-NB_Dis" departPos="free" departSpeed="max"/>

YAY!!!
flow_vph_1 being replaced  	<flow begin="1" end="900" route="ConcrseDR-Belmont-MemHll-WB-02-Distrib" vehsPerHour="6" type="US1-ALL-NB_Dis" departPos="free" departSpeed="max"/>



In [30]:
f.close()

In [11]:
cali_M_DF.iloc[1,0]

'ConcrseDR-Belmont-MemHll-EB-02-RouteProbe'

In [ ]:
readfromPATH = '/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/Belmont_AOI_git/Belmont_AOI_toolBOX/Belmount_AOI-caliborators-DB-V6-SUMO31-15min_RAW.xml'
        with open(readfromPATH) as f:
        #open('/Dropbox/Phd_R_Ms/PhD_Modeling_DB_GIT/Belmont_AOI_git/Belmont_AOI-runFILES/BMAOI_TRACI_DB.sumocfg') as f:
            termLIST = list()
            termcounter = 0
            for line in f:
                # print("SUMO_outPUT_PREFIX = ",SUMO_outPUT_PREFIX)
                # if SUMO_outPUT_PREFIX =="":
#                 routeProbe id="ConcrseDR-Belmont-MemHll-EB-02-RouteProbe" edge="-106455704#9"
                if '<routeProbe id="' in line:
                    counter += 1
#                     print(line)
                    routeProbe_ID_LINE = line
                    routeProbe_ID = re.search('<routeProbe id="(.*)" e',routeProbe_ID_LINE, re.IGNORECASE).group(1)
                    termLIST.append(re.search('<routeProbe id="(.*)" e',routeProbe_ID_LINE, re.IGNORECASE).group(1))
                    logger_TEMPDF.loc['routeProbe_ID',counter-1] = routeProbe_ID
#                     routeProbe_ID = termLIST[0]
                    # print("\nSUMO_outPUT_PREFIX = ",SUMO_outPUT_PREFIX,"\n")
                    termcounter +=1
                elif '<calibrator id=' in line:
                    calibrator_ID_LINE = line
                    calibrator_ID = re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(1)
                    termLIST.append(re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(1))
                    calibrator_ID_edge = re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(2)
                    termLIST.append(re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(2))
                    logger_TEMPDF.loc['calibrator_ID',counter-1] = calibrator_ID
                    logger_TEMPDF.loc['SUMO_edge_ID',counter-1] = calibrator_ID_edge
#                 elif '<flow begin="1"' in line:
# #                     print(line)
# #                     print(re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', line, re.IGNORECASE).group(3))
#                     flow_vph_1_ID_LINE = line
#                     flow_vph_1_ID = re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_1_ID_LINE, re.IGNORECASE).group(3)
#                     termLIST.append(re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_1_ID_LINE, re.IGNORECASE).group(3))
#                     logger_TEMPDF.loc['vph_start_1',counter-1] = flow_vph_1_ID
                elif '<flow begin="1"' in line:
                    flow_vph_1_ID_LINE = line
                    flow_vph_1_ID = re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_1_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="1" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_1_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_1', counter-1] = flow_vph_1_ID
                elif '<flow begin="901"' in line:
                    flow_vph_901_ID_LINE = line
                    flow_vph_901_ID = re.search('<flow begin="901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t',flow_vph_901_ID_LINE, re.IGNORECASE).group(3)
                    termLIST.append(re.search('<flow begin="901" end="(.*)" route="(.*)" vehsPerHour="(.*)" t', flow_vph_901_ID_LINE, re.IGNORECASE).group(3))
                    logger_TEMPDF.loc['vph_start_901', counter-1] = flow_vph_901_ID

In [ ]:
for coles in range(colsDF-1):
    print(worksheet_LIST[coles], "coles = ", coles, end='; ')
    for rowes in range(rowsDF):
        if coles == 0:
#             changein_ESAL_DF.loc[Belmont_AVEDic[int(rowes)-1],worksheet_LIST[coles]] = wb[worksheet_LIST[1]].cell(row=rowes, column=9).value
            for rowes2 in range(1,3+rowsDF):
                if wb[worksheet_LIST[coles]].cell(row=rowes2, column=1).value == Belmont_AVEDic[int(rowes)]:
                    changein_ESAL_DF.loc[Belmont_AVEDic[rowes],worksheet_LIST[coles]] = wb[worksheet_LIST[coles]].cell(row=rowes2, column=9).value
#                     print(rowes,"; ",rowes2,Belmont_AVEDic[int(rowes)]," = ",wb[worksheet_LIST[0]].cell(row=rowes2, column=9).value)
        else:
            for rowes2 in range(1,3+rowsDF):
                if wb[worksheet_LIST[0]].cell(row=rowes2, column=1).value == Belmont_AVEDic[int(rowes)]:

In [33]:
row_cali = cali_M_DF.shape[0]
col_cali = cali_M_DF.shape[1]
with open(caliXMLTESTPATH,'r') as f:
    capture = 0
    for line in f:
        if '<routeProbe id="' in line:
            routeProbe_ID_LINE = line
            routeProbe_ID = re.search('<routeProbe id="(.*)" e',routeProbe_ID_LINE, re.IGNORECASE).group(1)
            for cols in range(col_cali):
                if cali_M_DF.iloc[1,cols] == routeProbe_ID:
                    col_DF = cols
                    capture = 1
                    break
        if capture == 1:
            if '<calibrator id=' in line:
                calibrator_ID_LINE = line
                calibrator_ID = re.search('<calibrator id="(.*)" edge="(.*)" p',calibrator_ID_LINE, re.IGNORECASE).group(1)
                if calibrator_ID == cali_M_DF.iloc[0,col_DF]:
                    print("YAY!!!")
                else:
                    print("Shit...")
            elif '<flow begin="1"' in line:
                flow_vph_1_row = 3
                flow_vph_1_ID_LINE = line
                flow_vph_1_ID = "POO"#str(flow_vph_1_ID_LINE,cali_M_DF.iloc[flow_vph_1_row,col_DF])# * 1.33) # This needs tp be a function
                re.sub('vehsPerHour="(.*)" t','vehsPerHour=",flow_vph_1_ID," t', calibrator_ID_LINE, re.IGNORECASE)
                capture = 0
                print("flow_vph_1 being replaced ", line)
#         print(capture)



YAY!!!
flow_vph_1 being replaced  	<flow begin="1" end="900" route="ConcrseDR-Belmont-MemHll-EB-02-Distrib" vehsPerHour="5" type="US1-ALL-NB_Dis" departPos="free" departSpeed="max"/>

YAY!!!
flow_vph_1 being replaced  	<flow begin="1" end="900" route="ConcrseDR-Belmont-MemHll-WB-02-Distrib" vehsPerHour="6" type="US1-ALL-NB_Dis" departPos="free" departSpeed="max"/>

